In [ ]:
from qiskit_aer.noise import coherent_unitary_error
from qiskit import QuantumCircuit
import numpy as np

def create_coherent_depol_backend(coherent_error: float, depol_error: float):
    """Returns a noisy simulator backend with coherent error on Rz gates and depolarizing error
    This doesn't actually make sense in practice since Rz gates done in practice,
    but we're doing this for now since the Trotter circuit only has Rz and CX gates

    Args:
        coherent_error (float): Between 0 and 1. #0.01

    Returns:
        backend: A noisy simulator with coherent ZZ error
    """
    # Create the coherent error channel on 2Q gates
    # Copy-pasted from `test_ampcal1Q` in https://programtalk.com/python-more-examples/qiskit.providers.aer.noise.NoiseModel/
    err_unitary = np.zeros([2, 2], dtype=complex)
    for i in range(2):
        err_unitary[i, i] = np.cos(coherent_error)
        err_unitary[i, (i+1) % 2] = np.sin(coherent_error)
    err_unitary[0, 1] *= -1.0
    err_coherent = coherent_unitary_error(err_unitary).tensor(coherent_unitary_error(err_unitary))

    # We want more physical error model here, over-rotation on the target in CNOT. And do some test before doing it.



    # Create the depolarizing error channel
    err_depol = depolarizing_error(depol_error, 2) # error only on CNOT gates



    # Create a noise model from the error channels
    noisemodel = NoiseModel()
    err_total = err_coherent.compose(err_depol)
    noisemodel.add_all_qubit_quantum_error(err_total, ['cx'])

    err_single = coherent_unitary_error(err_unitary)
    noisemodel.add_all_qubit_quantum_error(err_single, ['rx'])

    backend = AerSimulator(noise_model=noisemodel)

    return backend